## Classiq Implementation For The Research Paper: Symmetry enhanced variational quantum spin eigensolver

In [45]:
# Handle imports
from classiq import * # Using Classiq Version 0.52.0 Release
import numpy as np
# import scipy as sp

In [46]:
# Circuit design variables
n_bits = 4
n_layers = 5
J = 1

# Dummy parameters for testing implementation of ciruit
theta_1 = np.pi
theta_2 = np.pi
theta_3 = np.pi
theta_4 = np.pi
theta_5 = np.pi
theta_6 = np.pi
theta_7 = np.pi

training_params = [theta_1,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7]

# Initialization circuit layer
@qfunc
def Init(q: QArray) -> None:
    # First, do PauliX on all qubits
    for i in range(n_bits):
        X(q[i])

    # Next, add the Hadamards and CNOT
    for i in range(0, n_bits, 2):
        hadamard_transform(q[i])
        CX(q[i],q[i + 1])


# N block function
@qfunc
def N_block(q0: QBit, q1: QBit, param: CReal) -> None:
    RZ(np.pi / 2, q1)

    CX(q1, q0)
     
    RZ(2 * param - np.pi/2, q0)
    RY(np.pi / 2 - 2 * param, q1)
    
    CX(q0, q1)

    RY(2 * param - np.pi/2, q1)

    CX(q1, q0)
    RZ(-np.pi / 2, q0)


# Main ansatz layer
@qfunc
def Sz_conserving_layer(q: QArray, params: CArray[CReal]) -> None:
    # Apply the N_block layer for entangling
    N_block(q[0],q[1], params[0])
    N_block(q[2],q[3], params[1])
    N_block(q[1],q[2], params[2])

    # Apply the Phase Gates
    for i in range(n_bits):
        PHASE(params[3+i], q[i])

# Function to build the Paulis string
def build_pauli_string(N: int, J: float) -> list[tuple[str, float]]:
    pauli_list = []
    
    # Loop over each neighboring pair of spins (i, i+1)
    for i in range(N-1):
        # Create Pauli strings for sigma_x, sigma_y, sigma_z interactions
        for pauli in ['X', 'Y', 'Z']:
            # Identity for other positions, Pauli for position i and i+1
            identity = ['I'] * N
            identity[i] = pauli
            identity[i+1] = pauli
            
            # Join Pauli string and add it to the list with coefficient J
            pauli_string = ''.join(identity)
            pauli_list.append((pauli_string, J))
    
    return pauli_list

# Main Program
@qfunc
def main():
    # Create the array of bits of proper size
    q = QArray("q")
    allocate(n_bits, q)
    
    # Do the initialization circuit
    Init(q)

    # Do n layers of the Sz_conserving Ansatz
    for _ in range(n_layers):
        Sz_conserving_layer(q,training_params)


In [47]:
# Simulation
mod = create_model(main)
qprog = synthesize(mod)
show(qprog)

/usr/bin/xdg-open: 882: x-www-browser: Permission denied
/usr/bin/xdg-open: 882: firefox: Permission denied
/usr/bin/xdg-open: 882: iceweasel: Permission denied
/usr/bin/xdg-open: 882: seamonkey: Permission denied
/usr/bin/xdg-open: 882: mozilla: Permission denied
/usr/bin/xdg-open: 882: epiphany: Permission denied
/usr/bin/xdg-open: 882: konqueror: Permission denied
/usr/bin/xdg-open: 882: chromium: Permission denied
/usr/bin/xdg-open: 882: chromium-browser: Permission denied
/usr/bin/xdg-open: 882: google-chrome: Permission denied
/usr/bin/xdg-open: 882: www-browser: Permission denied
/usr/bin/xdg-open: 882: links2: Permission denied
/usr/bin/xdg-open: 882: elinks: Permission denied
/usr/bin/xdg-open: 882: links: Permission denied
/usr/bin/xdg-open: 882: lynx: Permission denied
/usr/bin/xdg-open: 882: w3m: Permission denied
xdg-open: no method available for opening 'https://platform.classiq.io/circuit/c19169f1-4dd4-4436-ba20-72a3eb08cf86?version=0.53.0'
